In [ ]:
import pandas as pd
import seaborn as sns
import pickle
import numpy as np
import os
import time
import catboost as ctb

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

In [ ]:
%matplotlib inline

In [ ]:
f = open("D:\Doctorat\Programs\Jupyter Notebook\All_Results.txt", "w")
f.write("Circle_Radius,Delta_Time,Features_Index,Model,Precision,Recall,Accuracy,F1\n")
for circle_radius in range(50,60,10):
    for delta_time_index in range(1,15):
        if delta_time_index==1:
            delta_time=10
        if delta_time_index==2:
            delta_time=15
        if delta_time_index==3:
            delta_time=20
        if delta_time_index==4:
            delta_time=22.5
        if delta_time_index==5:
            delta_time=25
        if delta_time_index==6:
            delta_time=27.5
        if delta_time_index==7:
            delta_time=30
        if delta_time_index==8:
            delta_time=32.5
        if delta_time_index==9:
            delta_time=35
        if delta_time_index==10:
            delta_time=40
        if delta_time_index==11:
            delta_time=45
        if delta_time_index==12:
            delta_time=50
        if delta_time_index==13:
            delta_time=55
        if delta_time_index==14:
            delta_time=60
        file_name = "Final Summary - STAR - DXB - ({rd:.0f} NM) Revision 03 ({dt:.1f} minutes).csv"
        file_name=file_name.format(dt=delta_time,rd=circle_radius)        
        while os.path.isfile(file_name) == False:
            print(file_name+" does not exist yet.")
            time.sleep(10)    
        time.sleep(10) 
        print(file_name+" exists!")
        df = pd.read_csv(file_name,low_memory=False)
        file_name_f="Final Summary - STAR - DXB - ({rd:.0f} NM) Revision 03 ({dt:.1f} minutes).f"
        file_name_f=file_name_f.format(dt=delta_time,rd=circle_radius)
        column_1="TIME UTC - ENTRY {rd:.0f} NM RADIUS"
        column_1=column_1.format(rd=circle_radius)
        column_2="FLAG - ESTIMATED ENTRY {rd:.0f} NM RADIUS"
        column_2=column_2.format(rd=circle_radius)
        column_3="DISTANCE OF THE CLOSEST POINT TO {rd:.0f} NM RADIUS (NM)"
        column_3=column_3.format(rd=circle_radius)
        column_4="DELTA TIME ENTRY @ {rd:.0f} NM CIRCLE FROM PREVIOUS FLIGHT IN THE SAME QUADRANT (MIN.)"
        column_4=column_4.format(rd=circle_radius)
        df.drop(columns=["DATE","ICAO24","FILE","PROCEDURE","FLIGHT NUMBER","STA","DELTA STA (MIN.)","TOUCHDOWN TIME",column_1,column_2,column_3,"RECOMPUTED TOUCHDOWN TIME","MINIMUM DISTANCE TO RUNWAY THRESHOLD (NM)","DELTA TOUCHDOWN TIME @ SAME RUNWAY (PREVIOUS LANDING) (MIN.)",column_4,"SPILLAGE\n(0 - NO/\n1 - YES)?","LONGITUDE - HOLDING POINT","LATITUDE - HOLDING POINT","DISTANCE FROM HOLDING POINT TO DXB (NM)","DAY OF WEEK"]).reset_index(drop=True).to_feather(file_name_f)
        df = pd.read_feather(file_name_f)
        df=df.replace("12L",0)
        df=df.replace("12R",0)
        df=df.replace("30L",1)
        df=df.replace("30R",1)
        df=df.replace("PUVAL",0)
        df=df.replace("VUTEB",1)
        df=df.replace("DATOB",1)
        df=df.replace("LORID",2)
        df=df.replace("IMPED",3)
        df=df.replace("L",0)
        df=df.replace("M",1)
        df=df.replace("H",2)
        df=df.replace("J",3)
        for features_index in range(1,4):
            feature_1="LANDING RUNWAY"
            feature_2="LONGITUDE - ENTRY "+str(circle_radius)+" NM RADIUS"
            feature_3="LATITUDE - ENTRY "+str(circle_radius)+" NM RADIUS"
            feature_4="NUMBER OF AIRCRAFT ENTERING THE "+str(circle_radius)+" NM RADIUS DURING THE {dt:.1f} MINUTES PERIOD BEFORE THE INTERCEPTION OF THE CIRCLE"
            feature_4=feature_4.format(dt=delta_time) 
            feature_5="NUMBER OF AIRCRAFT TAKING-OFF DURING THE {dt:.1f} MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE"
            feature_5=feature_5.format(dt=delta_time) 
            feature_6="TOTAL NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE"
            feature_7="NUMBER OF AIRCRAFT TOUCHING DOWN DURING THE {dt:.1f} MINUTES PERIOD AFTER THE INTERCEPTION OF THE CIRCLE"
            feature_7=feature_7.format(dt=delta_time)
            feature_8="WAKE TURBULENCE CATEGORY"
            feature_9="ALTITUDE - ENTRY "+str(circle_radius)+" NM RADIUS (M)"
            feature_10="TRUE SPEED - ENTRY "+str(circle_radius)+" NM RADIUS (M/S)"
            feature_11="METAR VISIBILITY (KM)"
            if features_index==1:
                features = [feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_11]
                features_text="WITH TOTAL NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE AND OTHER PARAMETERS"
            if features_index==2:
                features = [feature_1,feature_2,feature_3,feature_4,feature_5,feature_7,feature_8,feature_11]
                features_text="WITHOUT TOTAL NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE AND OTHER PARAMETERS"
            if features_index==3:
                features = [feature_1,feature_2,feature_3,feature_6]
                features_text="ONLY WITH TOTAL NUMBER OF AIRCRAFT FLYING INSIDE THE "+str(circle_radius)+" NM RADIUS AT THE INTERCEPTION OF THE CIRCLE"
            targets = ["FLAG HOLDING"]
            all_items = features + targets 
            df = df[df["QUADRANT"].str.contains("-") == False]
            df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)
            df_train_x = df_train.filter(items=features)
            df_train_y = df_train.filter(items=targets)
            df_test_x = df_test.filter(items=features)
            df_test_y = df_test.filter(items=targets)
            scaler=MinMaxScaler()
            scaler.fit(df_train_x)
            df_train_x=scaler.transform(df_train_x)
            df_test_x=scaler.transform(df_test_x)
            df_train_y=df_train_y.values.flatten()
            for model_index in range(1,7):
                if model_index==1:
                    model=DecisionTreeClassifier(max_depth=12)
                    model_text="Decision Tree"
                if model_index==2:
                    model=KNeighborsClassifier(n_neighbors=10,weights='distance',leaf_size=60)
                    model_text="K Neighbors"
                if model_index==3:
                    model=RandomForestClassifier(n_estimators=100,max_depth=12)
                    model_text="Random Forest"
                if model_index==4:                
                    model=MLPClassifier(max_iter=500,activation='relu',solver='adam',random_state=1)        
                    model_text="Multi Layer Perceptron"
                if model_index==5:                
                    model=GradientBoostingClassifier()        
                    model_text="Gradient Boosting"                    
                if model_index==6:
                    model=ctb.CatBoostClassifier(iterations=1000,learning_rate=1,depth=2)                    
                    model_text="CatBoost"                    
                print("================================================================================")
                print("Circle Radius =",circle_radius," NM")                
                print("Delta Time =",delta_time," minutes")
                print(features_text)
                print(model_text)            
                model.fit(df_train_x, df_train_y)
                y_pred=model.predict(df_test_x)
                precision=precision_score(df_test_y, y_pred)
                recall=recall_score(df_test_y, y_pred)
                accuracy=accuracy_score(df_test_y, y_pred)
                f1=f1_score(df_test_y, y_pred)            
                print("Precision =",precision)
                print("Recall =",recall)
                print("Accuracy =",accuracy)
                print("F1 =",f1)
                line=str(circle_radius)+","+str(delta_time)+","+features_text+","+model_text+","+str(precision)+","+str(recall)+","+str(accuracy)+","+str(f1)            
                f.write(line+"\n")
f.close